In [1]:
import numpy as np
from numpy.linalg import norm
import pickle
from tqdm.notebook import tqdm
import os
import time
import math
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
MODEL = ResNet50(weights='imagenet',
                         include_top=False,
                         input_shape=(224, 224, 3),
                        pooling='max')

In [4]:
EXTENSIONS = {'.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG'}

def get_file_list(root_dir):
    return [os.path.join(root, filename)
            for root, _, filenames in os.walk(root_dir)
            for filename in filenames
            if any(filename.endswith(ext) for ext in EXTENSIONS)
            if os.path.exists(os.path.join(root, filename))]


ROOT_DIR = './datasets/caltech101'
filenames = sorted(get_file_list(ROOT_DIR))
print('Количество изображений:', len(filenames))


Количество изображений: 8677


In [5]:
# BATCH_SIZE партия изображений для одновременного извлечения признаков
# datagen предобрабатывает (в т.ч. нормализует) изображения перед подачей в модель
# generator генератор данных, который будет загружать изображения из директории root_dir
## Параметр class_mode=None означает, что мы не используем информацию о классах изображений, 
## а просто извлекаем признаки. shuffle=False означает, что порядок изображений не будет перемешиваться
## для того чтобы сохранить соответствие между изображениями и их признаками
# NUM_IMAGES общее количество изображений в наборе данных
# NUM_EPOCHS количество итераций, необходимых для обработки всех изображений, учитывая BATCH_SIZE
# feature_list модель использует generator для извлечения признаков из всех изображений и сохраняет в список

with tf.device('/gpu:0'):
    BATCH_SIZE = 128
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)

    generator = datagen.flow_from_directory(ROOT_DIR,
                                            target_size=(224, 224),
                                            class_mode=None,
                                            shuffle=False)

    NUM_IMAGES = len(generator.filenames)
    NUM_EPOCHS = int(math.ceil(NUM_IMAGES / BATCH_SIZE))

    start_time = time.time()
    feature_list = []
    feature_list = MODEL.predict(generator, NUM_EPOCHS)
    end_time = time.time()

Found 8677 images belonging to 101 classes.


In [6]:
for i, features in enumerate(feature_list):
    feature_list[i] = features / norm(features)

feature_list = feature_list.reshape(len(feature_list), -1)

print("Num images   = ", len(generator.classes))
print("Shape of feature_list = ", feature_list.shape)
print("Time taken in sec = ", end_time - start_time)

Num images   =  8677
Shape of feature_list =  (8677, 2048)
Time taken in sec =  26.67900013923645


In [7]:
filenames = [ROOT_DIR + '/' + s for s in generator.filenames]

In [8]:
pickle.dump(generator.classes, open('./data/class_ids-caltech101.pickle',
                                    'wb'))
pickle.dump(filenames, open('./data/filenames-caltech101.pickle', 'wb'))
pickle.dump(
    feature_list,
    open('./data/features-caltech101-' + MODEL.name + '.pickle', 'wb'))